2.2. Data Preprocessing

So far, we have been working with synthetic data that arrived in ready-made tensors. However, to apply deep learning in the wild we must extract messy data stored in arbitrary formats, and preprocess it to suit our needs. Fortunately, the pandas library can do much of the heavy lifting. This section, while no substitute for a proper pandas tutorial, will give you a crash course on some of the most common routines.

In [208]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')

In [209]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000


In [210]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       NaN           False          True
1       2.0           False          True
2       4.0            True         False
3       NaN           False          True


In [211]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0           False          True
1       2.0           False          True
2       4.0            True         False
3       3.0           False          True


In [212]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))
X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

2.2.5. Exercises

    1. Try loading datasets, e.g., Abalone from the UCI Machine Learning Repository and inspect their properties. What fraction of them has missing values? What fraction of the variables is numerical, categorical, or text?

In [213]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
abalone = fetch_ucirepo(id=1) 
abalone

{'data': {'ids': None,
  'features':      Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  \
  0      M   0.455     0.365   0.095        0.5140          0.2245   
  1      M   0.350     0.265   0.090        0.2255          0.0995   
  2      F   0.530     0.420   0.135        0.6770          0.2565   
  3      M   0.440     0.365   0.125        0.5160          0.2155   
  4      I   0.330     0.255   0.080        0.2050          0.0895   
  ...   ..     ...       ...     ...           ...             ...   
  4172   F   0.565     0.450   0.165        0.8870          0.3700   
  4173   M   0.590     0.440   0.135        0.9660          0.4390   
  4174   M   0.600     0.475   0.205        1.1760          0.5255   
  4175   F   0.625     0.485   0.150        1.0945          0.5310   
  4176   M   0.710     0.555   0.195        1.9485          0.9455   
  
        Viscera_weight  Shell_weight  
  0             0.1010        0.1500  
  1             0.0485        0.0700  
  2  

In [214]:
abalone.metadata

{'uci_id': 1,
 'name': 'Abalone',
 'repository_url': 'https://archive.ics.uci.edu/dataset/1/abalone',
 'data_url': 'https://archive.ics.uci.edu/static/public/1/data.csv',
 'abstract': 'Predict the age of abalone from physical measurements',
 'area': 'Biology',
 'tasks': ['Classification', 'Regression'],
 'characteristics': ['Tabular'],
 'num_instances': 4177,
 'num_features': 8,
 'feature_types': ['Categorical', 'Integer', 'Real'],
 'demographics': [],
 'target_col': ['Rings'],
 'index_col': None,
 'has_missing_values': 'no',
 'missing_values_symbol': None,
 'year_of_dataset_creation': 1994,
 'last_updated': 'Mon Aug 28 2023',
 'dataset_doi': '10.24432/C55C7W',
 'creators': ['Warwick Nash',
  'Tracy Sellers',
  'Simon Talbot',
  'Andrew Cawthorn',
  'Wes Ford'],
 'intro_paper': None,
 'additional_info': {'summary': 'Predicting the age of abalone from physical measurements.  The age of abalone is determined by cutting the shell through the cone, staining it, and counting the number of r

In [215]:
abalone.data.features

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550
...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960


In [216]:
abalone.data.features.isnull().sum(axis = 1) 
# axis is the dimension to be collapsed into a single value. 
# "axis = 0" ==> rows ==> collapse all rows leaving only 1 value (sum) per column. 
# ex:  
#      a b c d e  =>  a b c d e
#    x 0 1 1 2 2      2 3 5 6 8 
#    y 1 1 2 2 3
#    z 1 1 2 2 3
# 
# "axis = 1" ==> columns ==> collapse all columns leaving only 1 value (sum) per row. 
# ex:
#      a b c d e  =>  x 7
#    x 0 1 1 2 2      y 9
#    y 1 1 2 2 3      z 9
#    z 1 1 2 2 3
# 


0       0
1       0
2       0
3       0
4       0
       ..
4172    0
4173    0
4174    0
4175    0
4176    0
Length: 4177, dtype: int64

In [217]:
abalone.data.features.dropna(axis=1, thresh=min(data.count(axis=0))+1)

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550
...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960


In [218]:
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
wine_quality.data.features.isnull().sum(axis = 0)


fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
dtype: int64

In [219]:
wine_quality.variables

,name,role,type,demographic,description,units,missing_values
0,fixed_acidity,Feature,Continuous,None,None,None,no
1,volatile_acidity,Feature,Continuous,None,None,None,no
2,citric_acid,Feature,Continuous,None,None,None,no
3,residual_sugar,Feature,Continuous,None,None,None,no
4,chlorides,Feature,Continuous,None,None,None,no
5,free_sulfur_dioxide,Feature,Continuous,None,None,None,no
6,total_sulfur_dioxide,Feature,Continuous,None,None,None,no
7,density,Feature,Continuous,None,None,None,no
8,pH,Feature,Continuous,None,None,None,no
9,sulphates,Feature,Continuous,None,None,None,no


In [220]:
wine_quality.variables.value_counts("type")

type
Continuous     11
Categorical     1
Integer         1
Name: count, dtype: int64

In [221]:
pd.unique(wine_quality.variables["type"])

array(['Continuous', 'Integer', 'Categorical'], dtype=object)

In [222]:
len(pd.unique(wine_quality.variables["type"]))

3

In [223]:
wine_quality.data.features.isnull().sum(axis = 0)

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
dtype: int64

In [224]:
heart_disease = fetch_ucirepo(id=45)
heart_disease.data.features.isnull().sum(axis = 0)

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
dtype: int64

In [225]:
heart_disease.data.features.isnull().sum(axis = 0).sum(axis = 0)
print("percent of missing values:", (heart_disease.data.features.isnull().sum(axis = 0).sum(axis = 0) / (heart_disease.data.features.shape[0] * heart_disease.data.features.shape[1])) * 100)

percent of missing values: 0.15232292460015232


In [226]:
heart_disease.variables

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,years,no
1,sex,Feature,Categorical,Sex,None,None,no
2,cp,Feature,Categorical,None,None,None,no
3,trestbps,Feature,Integer,None,resting blood pressure (on admission to the ho...,mm Hg,no
4,chol,Feature,Integer,None,serum cholestoral,mg/dl,no
5,fbs,Feature,Categorical,None,fasting blood sugar > 120 mg/dl,None,no
6,restecg,Feature,Categorical,None,None,None,no
7,thalach,Feature,Integer,None,maximum heart rate achieved,None,no
8,exang,Feature,Categorical,None,exercise induced angina,None,no
9,oldpeak,Feature,Integer,None,ST depression induced by exercise relative to ...,None,no


In [227]:
heart_disease.variables.value_counts("type")

type
Categorical    7
Integer        7
Name: count, dtype: int64

In [228]:
heart_disease.data.features

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0


A:
- abalone has no missing values, and of the 8 features, 7 are numeric and 1 is categorical. (the target is also categorical)

- wine_quality has no missing values, and of the 13 features, all are numerical and none are categorical. (the target is categorical)

- heart_disease has 6 missing values out of a total of 3939 variables (0.152%), and of 13 features, 7 are numerical and 6 are categorical. (the target is also categorical)

2.2.5. Exercises

    2. Try indexing and selecting data columns by name rather than by column number. The pandas documentation on indexing has further details on how to do this.

In [229]:
abalone.data.features[["Sex", "Length", "Height"]][:5]

,Sex,Length,Height
0,M,0.455,0.095
1,M,0.350,0.090
2,F,0.530,0.135
3,M,0.440,0.125
4,I,0.330,0.080


In [230]:
wine_quality.data.features[["fixed_acidity", "volatile_acidity", "citric_acid"]][:5]

,fixed_acidity,volatile_acidity,citric_acid
0,7.4,0.70,0.00
1,7.8,0.88,0.00
2,7.8,0.76,0.04
3,11.2,0.28,0.56
4,7.4,0.70,0.00


In [231]:
heart_disease.data.features[["age", "sex", "cp"]][:5]

,age,sex,cp
0,63,1,1
1,67,1,4
2,67,1,4
3,37,1,3
4,41,0,2


2.2.5. Exercises

    3. How large a dataset do you think you could load this way? What might be the limitations? Hint: consider the time to read the data, representation, processing, and memory footprint. Try this out on your laptop. What happens if you try it out on a server?

A: Pandas DataFrames are loaded into RAM, so it depends on how much RAM is available, how fast the RAM is, any bottlenecks, etc.

2.2.5. Exercises

    4. How would you deal with data that has a very large number of categories? What if the category labels are all unique? Should you include the latter?

A: To deal with a large number of categories, you could try to eliminate any superfluous categories, or categories that may not have data that is significant. Another method would be to try to find high correlations between two or more categories and combine them.
If the category labels are all unique, meaning every value in a column is unique, then the column doesn't carry any useful information and can be dropped.

2.2.5. Exercises

    5. What alternatives to pandas can you think of? How about loading NumPy tensors from a file? Check out Pillow, the Python Imaging Library.

A:
- PIL library can be used to open an image file as an Image object.
    - the Image object can be interpreted as an array, as per the NumPy asarray() method.
- dask adds parallelization to python. still uses standard dataframes, numpy, etc.
- pyspark
- polars
- there are a ton. here's a (probably) non-exhaustive list:
https://github.com/jcmkk3/awesome-dataframes